In [1]:
# best way to compute the precision would be to estimate the confusion matrix and then make the calculation
import torch
import numpy as np

In [2]:
correct_labels = torch.LongTensor([0, 1, 2, 1, 1, 3, 1, 2, 3, 0, 2, 3, 2, 0, 3, 0])

In [3]:
predict_labels = torch.LongTensor([0, 3, 0, 2, 3, 2, 0, 3, 2, 1, 3, 1, 1, 2, 1, 0])

In [36]:
def get_cnf_matrix(y_true, y_pred):
    
    sample_weight = torch.ones(y_true.shape[0], dtype=torch.int64)
    
    labels = torch.unique(torch.cat((y_true, y_pred)))
    
    label_to_ind = {y.item(): x for x, y in enumerate(labels)}
    
    n_labels = labels.shape.numel()
    _y_pred = y_pred.new_tensor([label_to_ind.get(x.item(), n_labels+1) for x in y_pred])
    _y_true = y_true.new_tensor([label_to_ind.get(x.item(), n_labels+1) for x in y_true])
    
    ind = torch.logical_and(_y_pred < n_labels, y_true < n_labels)
    
    _y_pred = _y_pred[ind]
    _y_true = _y_true[ind]
    sample_weight = sample_weight[ind]
    
    cm_ind = torch.cat((_y_true.view(1, -1), _y_pred.view(1, -1)), dim=0)
    
    cm = torch.sparse.LongTensor(cm_ind, sample_weight, torch.Size([n_labels, n_labels]))
    
    return cm.to_dense()

In [32]:
ll = torch.cat((correct_labels.view(1, -1), predict_labels.view(1, -1)), dim=0)

In [33]:
ll.shape

torch.Size([2, 16])

In [14]:
torch.ones(4)

tensor([1., 1., 1., 1.])

In [41]:
correct_labels < 4

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [15]:
np.ones(4)

array([1., 1., 1., 1.])

In [16]:
correct_labels.shape[0]

16

In [42]:
cm = get_cnf_matrix(correct_labels, predict_labels)
cm

tensor([[2, 1, 1, 0],
        [1, 0, 1, 2],
        [1, 1, 0, 2],
        [0, 2, 2, 0]])

In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
y_true = correct_labels.numpy()
y_pred = predict_labels.numpy()

In [40]:
confusion_matrix(y_true, y_pred)

array([[2, 1, 1, 0],
       [1, 0, 1, 2],
       [1, 1, 0, 2],
       [0, 2, 2, 0]])

In [51]:
# calculate the precision tp/(tp+fp)
# true positives are the diagonals
tp = torch.diagonal(cm)
tp

tensor([2, 0, 0, 0])

In [52]:
# false positives are the 
fp = torch.sum(cm, dim=0) - tp
fp

tensor([2, 4, 4, 4])

In [61]:
torch

tensor([[2, 1, 1, 0],
        [1, 0, 1, 2],
        [1, 1, 0, 2],
        [0, 2, 2, 0]])

In [55]:
# unweighted precision = 
prec = torch.true_divide(tp, tp+fp)
prec

tensor([0.5000, 0.0000, 0.0000, 0.0000])

In [56]:
# to get the weights of each label, we'd need how many of those elements there were
weights = torch.sum(cm, dim=1)
weights

tensor([4, 4, 4, 4])

In [57]:
elem_count = torch.sum(weights)

In [59]:
weighted_prec = torch.avg_pool1d

In [60]:
weighted_prec

tensor([0.1250, 0.0000, 0.0000, 0.0000])

In [62]:
prob_estimates = torch.rand(5, 10)

In [67]:
prob_estimates = torch.FloatTensor([[0.1270, 0.3182, 0.7561, 0.2694, 0.8137, 0.4122, 0.0514, 0.0417, 0.2929,
         0.3658],
        [0.2220, 0.6216, 0.3743, 0.3379, 0.8600, 0.2977, 0.5080, 0.5919, 0.3067,
         0.7062],
        [0.5227, 0.5934, 0.6640, 0.9494, 0.9338, 0.0082, 0.6514, 0.4110, 0.0067,
         0.7598],
        [0.1915, 0.4978, 0.8259, 0.1727, 0.1590, 0.3756, 0.2304, 0.7274, 0.3615,
         0.2914],
        [0.5394, 0.1803, 0.6762, 0.0320, 0.5921, 0.3524, 0.3931, 0.8371, 0.7799,
         0.4619]])

In [69]:
torch.argmax(prob_estimates, dim=1)

tensor([4, 4, 3, 2, 7])

In [70]:
# target tensor, with this target we expect a top-5 accuracy of 60%
y_target = torch.LongTensor([2, 0, 1, 8, 7]) 

In [71]:
class_labels = torch.LongTensor(range(10))

In [76]:
sorted_prob = torch.argsort(prob_estimates, dim=1, descending=True)

In [77]:
sorted_prob

tensor([[4, 2, 5, 9, 1, 8, 3, 0, 6, 7],
        [4, 9, 1, 7, 6, 2, 3, 8, 5, 0],
        [3, 4, 9, 2, 6, 1, 0, 7, 5, 8],
        [2, 7, 1, 5, 8, 9, 6, 0, 3, 4],
        [7, 8, 2, 4, 0, 9, 6, 5, 1, 3]])

In [98]:
top_n = sorted_prob[:, :1]

In [99]:
top_n

tensor([[4],
        [4],
        [3],
        [2],
        [7]])

In [100]:
top_n.shape

torch.Size([5, 1])

In [103]:
y_target = torch.LongTensor([2, 0, 1, 8, 7])

In [104]:
top_n == y_target.view(1, -1)

tensor([[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [ True, False, False, False, False],
        [False, False, False, False,  True]])

In [105]:
pred = top_n == y_target.view(-1, 1)

In [106]:
pred.shape[0]

5

In [107]:
torch.sum(pred)

tensor(1)

In [89]:
est = np.array([[0.1270, 0.3182, 0.7561, 0.2694, 0.8137, 0.4122, 0.0514, 0.0417, 0.2929,
         0.3658],
        [0.2220, 0.6216, 0.3743, 0.3379, 0.8600, 0.2977, 0.5080, 0.5919, 0.3067,
         0.7062],
        [0.5227, 0.5934, 0.6640, 0.9494, 0.9338, 0.0082, 0.6514, 0.4110, 0.0067,
         0.7598],
        [0.1915, 0.4978, 0.8259, 0.1727, 0.1590, 0.3756, 0.2304, 0.7274, 0.3615,
         0.2914],
        [0.5394, 0.1803, 0.6762, 0.0320, 0.5921, 0.3524, 0.3931, 0.8371, 0.7799,
         0.4619]])

In [90]:
y_target = np.array([2, 0, 1, 8, 7]) 

In [91]:
np_sorted_prob = np.argsort(-est, axis=1)

In [92]:
np_sorted_prob

array([[4, 2, 5, 9, 1, 8, 3, 0, 6, 7],
       [4, 9, 1, 7, 6, 2, 3, 8, 5, 0],
       [3, 4, 9, 2, 6, 1, 0, 7, 5, 8],
       [2, 7, 1, 5, 8, 9, 6, 0, 3, 4],
       [7, 8, 2, 4, 0, 9, 6, 5, 1, 3]])

In [93]:
np_top_n = np_sorted_prob[:, :5]

In [94]:
np_top_n

array([[4, 2, 5, 9, 1],
       [4, 9, 1, 7, 6],
       [3, 4, 9, 2, 6],
       [2, 7, 1, 5, 8],
       [7, 8, 2, 4, 0]])

In [95]:
np_top_n == y_target.reshape(-1, 1)

array([[False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False,  True],
       [ True, False, False, False, False]])

In [108]:
torch.zeros((5, 5))

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [109]:
torch.sum(torch.ones(5, 5))

tensor(25.)

In [110]:
import tempfile

In [111]:
import os

In [112]:
os.path.join("cool", "yess")

'cool/yess'